In [28]:
#Dependencies
import pandas as pd
import urllib.parse
from io import StringIO
import matplotlib.pyplot as plt
import glob
import datetime
import calendar
import time
import os
from data_utils import establish_nys_session
import warnings
warnings.filterwarnings('ignore')

Read the CSVs into a pandas df

In [59]:
folder_path = 'Ressources/'
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

dataframes = []  # List to hold all dataframes

for file in files:
    file_path = os.path.join(folder_path, file)  # Full path to file
    df = pd.read_csv(file_path)  # Read the file into a DataFrame
    dataframes.append(df)  # Append the DataFrame to the list
    csv_files = glob.glob('path_to_your_directory/*.csv')

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Concatenate all DataFrames in the list into one DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df

,date,station_complex,borough,payment_method,total_ridership,lat,lng
0,2022-02-01,103 St (1),Manhattan,metrocard,5021,40.799446,-73.968376
1,2022-02-01,103 St (6),Manhattan,metrocard,5711,40.790600,-73.947479
2,2022-02-01,"103 St (C,B)",Manhattan,metrocard,1624,40.796093,-73.961456
3,2022-02-01,103 St-Corona Plaza (7),Queens,metrocard,9449,40.749866,-73.862701
4,2022-02-01,104 St (A),Queens,metrocard,835,40.681709,-73.837685
...,...,...,...,...,...,...,...
671471,2024-04-05,WTC Cortlandt (1),Manhattan,omny,6646,40.711834,-74.012190
671472,2024-04-05,York St (F),Brooklyn,metrocard,3884,40.701397,-73.986750
671473,2024-04-05,York St (F),Brooklyn,omny,6378,40.701397,-73.986750
671474,2024-04-05,Zerega Av (6),Bronx,metrocard,1032,40.836487,-73.847040


What are the most popular stations per Year?

#Revenue Per Borough from 2022 - Present